In [62]:
# @title Upload CSV
import pandas as pd
import random

hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')

In [63]:
# @title Gabungkan data_dosen dengan dosen
merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")
print(f"Total data dalam merged_df: {len(merged_df)}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282
Total data dalam merged_df: 107


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [64]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    for id_counter, (hari, ruang, jam) in enumerate(
        [(h, r, j) for h in hari_list for r in ruang_list for j in jam_list], start=1
    ):
        wolf.append({
            "id_slot": id_counter,
            "mata_kuliah": None,
            "dosen": None,
            "ruang": ruang,
            "hari": hari,
            "jam_mulai": jam['jam_awal'],
            "jam_selesai": jam['jam_akhir'],
            "kelas": None,
            "sks": None,
            "metode": None
        })
    return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [65]:
import random
from collections import defaultdict
from datetime import datetime

# Fungsi bantu untuk konversi waktu ke menit
def time_to_minutes(t):
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        dt = datetime.strptime(t, "%H:%M")
    return dt.hour * 60 + dt.minute

# Fungsi bantu untuk cek tumpang tindih waktu (overlap)
def is_overlap(new_block, allocated_blocks):
    new_start = time_to_minutes(new_block[0]['jam_mulai'])
    new_end = time_to_minutes(new_block[-1]['jam_selesai'])
    for (start, end) in allocated_blocks:
        # Jika ada tumpang tindih, return True
        if not (new_end <= start or new_start >= end):
            return True
    return False

# Fungsi bantu untuk mengecek apakah blok slot merupakan slot boundary (awal atau akhir hari) di ruangan yang sama
def is_boundary_block(block, schedule):
    hari = block[0]['hari']
    ruang = block[0]['ruang']
    # Jika perkuliahan online, lewati pengecekan boundary
    if ruang == "Online":
        return True
    # Filter seluruh slot pada hari dan ruangan yang sama
    slots_in_day = [s for s in schedule if s['hari'] == hari and s['ruang'] == ruang]
    # Urutkan berdasarkan waktu mulai
    slots_in_day.sort(key=lambda s: time_to_minutes(s['jam_mulai']))
    earliest = slots_in_day[0]['jam_mulai']
    latest = slots_in_day[-1]['jam_selesai']
    # Jika blok diawali slot paling awal atau diakhiri slot paling akhir
    return (block[0]['jam_mulai'] == earliest) or (block[-1]['jam_selesai'] == latest)

def create_random_schedule():
    # Ambil slot dari fungsi slot_generator() (pastikan fungsi ini sudah didefinisikan)
    schedule = slot_generator()
    # Acak urutan data matakuliah dari merged_df (pastikan merged_df sudah didefinisikan)
    merged_shuffled = merged_df.sample(frac=1).iterrows()

    courses_inserted = 0  # Menghitung jumlah mata kuliah yang berhasil ditempatkan

    # Struktur untuk tracking alokasi, guna mengecek konflik
    room_allocations = defaultdict(list)    # key: (ruang, hari) -> list of (start, end)
    teacher_allocations = defaultdict(list)  # key: (dosen, hari) -> list of (start, end)
    class_allocations = defaultdict(list)    # key: (kelas, hari) -> list of (start, end)

    for idx, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']

        # Dapatkan daftar semua posisi yang mungkin untuk blok sepanjang sks
        possible_positions = list(range(len(schedule) - sks + 1))
        random.shuffle(possible_positions)
        placed = False

        # Variabel untuk menampung blok kandidat yang tidak tumpang tindih
        candidate_blocks = []

        for i in possible_positions:
            block = schedule[i:i+sks]

            # Pastikan seluruh slot dalam blok masih kosong
            if not all(slot['mata_kuliah'] is None for slot in block):
                continue

            # Validasi: seluruh slot dalam blok harus berada pada hari yang sama
            if not all(slot['hari'] == block[0]['hari'] for slot in block):
                continue

            # Untuk perkuliahan offline, seluruh slot harus berada pada ruangan yang sama
            if metode != 'Online' and not all(slot['ruang'] == block[0]['ruang'] for slot in block):
                continue

            hari = block[0]['hari']
            ruang = "Online" if metode == 'Online' else block[0]['ruang']

            # Tentukan interval waktu blok
            start_time = time_to_minutes(block[0]['jam_mulai'])
            end_time = time_to_minutes(block[-1]['jam_selesai'])
            time_block = (start_time, end_time)

            # Cek konflik dosen: dosen tidak boleh mengajar di interval yang tumpang tindih
            if is_overlap(block, teacher_allocations[(dosen, hari)]):
                continue

            # Untuk offline, cek konflik ruangan
            if metode != 'Online' and is_overlap(block, room_allocations[(ruang, hari)]):
                continue

            # Jika sudah ada jadwal untuk kelas di hari itu, tidak perlu enforce boundary;
            # namun jika ini adalah jadwal pertama untuk kelas di hari tersebut, usahakan slot boundary (jika memungkinkan)
            kelas_already = len(class_allocations[(kelas, hari)]) > 0

            candidate_blocks.append((block, time_block, kelas_already))

        # Jika ada kandidat, prioritaskan blok yang memenuhi boundary constraint untuk kelas (jika kelas belum dijadwalkan hari itu)
        selected_block = None
        if candidate_blocks:
            # Filter kandidat yang memenuhi boundary jika kelas belum terjadwal hari itu
            boundary_candidates = [b for b, t, already in candidate_blocks if not already and is_boundary_block(b, schedule)]
            if boundary_candidates:
                selected_block = boundary_candidates[0]
            else:
                # Jika tidak ada kandidat boundary atau kelas sudah memiliki jadwal hari itu, ambil kandidat pertama
                selected_block = candidate_blocks[0][0]

        if selected_block is None:
            print(f"Gagal menempatkan: {mata_kuliah} - {dosen} - {kelas}")
        else:
            # Update tiap slot dalam blok dengan data mata kuliah
            for slot in selected_block:
                slot.update({
                    "mata_kuliah": mata_kuliah,
                    "dosen": dosen,
                    "kelas": kelas,
                    "sks": sks,
                    "metode": metode,
                    "ruang": "Online" if metode == 'Online' else slot['ruang']
                })

            # Update struktur tracking untuk konflik
            hari = selected_block[0]['hari']
            ruang = "Online" if metode == 'Online' else selected_block[0]['ruang']
            start_time = time_to_minutes(selected_block[0]['jam_mulai'])
            end_time = time_to_minutes(selected_block[-1]['jam_selesai'])
            time_block = (start_time, end_time)

            # Untuk offline, update alokasi ruang
            if metode != 'Online':
                room_allocations[(ruang, hari)].append(time_block)
            teacher_allocations[(dosen, hari)].append(time_block)
            class_allocations[(kelas, hari)].append(time_block)

            placed = True
            courses_inserted += 1

    total_courses = len(merged_df)
    print(f"\nTotal mata kuliah yang berhasil dimasukkan: {courses_inserted} / {total_courses}")
    return schedule

# Contoh penggunaan:
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for schedule in population:
    for slot in schedule:
        print(slot)



Total mata kuliah yang berhasil dimasukkan: 107 / 107
{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 6, 'mata_kuliah

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
dosen tidak boleh mengajar mata kuliah/kelas berbeda pada jam yang sama
ruang kelas tidak boleh digunakan lebih dari 1 kelas/dosen
3 sks membutuhkan 3 slot waktu
ruangan antar kelas harus sama
slot waktu harus berurutan

Konstrain ringan:
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [82]:
def calculate_fitness(schedule):
    max_score = 100
    penalty = 0

    teacher_conflicts = []         # Hard constraint: dosen mengajar dua kelas bersamaan
    room_conflicts = []            # Hard constraint: ruangan dipakai lebih dari satu kelas bersamaan

    # Tambahan: conflict preferensi dosen dan soft constraint hari Sabtu
    teacher_preference_conflicts = []  
    saturday_conflicts = []       

    # Tracking hard constraint
    teacher_intervals = defaultdict(list)  # key: (dosen, hari) -> list of (start, end, id_slot)
    room_intervals = defaultdict(list)       # key: (ruang, hari) -> list of (start, end, id_slot)
    course_groups = defaultdict(list)

    # Kumpulkan data dari tiap slot yang terisi
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue

        day = slot['hari']
        teacher = slot['dosen']
        room = slot['ruang']
        kelas = slot['kelas']
        id_slot = slot['id_slot']
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        teacher_intervals[(teacher, day)].append((start, end, id_slot))
        if room != "Online":
            room_intervals[(room, day)].append((start, end, id_slot))
        
        # Bangun kelompok untuk validasi SKS
        key = (slot['mata_kuliah'], kelas, day, slot['sks'])
        course_groups[key].append(slot)
        
        # Soft constraint: cek konflik preferensi dosen
        if str(teacher) in ["Ardiansyah, Dr., S.T., M.Cs."]:
            if start <= 720:
                teacher_preference_conflicts.append({
                    'teacher': teacher,
                    'id_slot': id_slot,
                    'jam': slot['jam_mulai'],
                    'message': "Hanya ingin dijadwalkan pada jam 12 atau lebih"
                })
                penalty += 0.5

        # Soft constraint: setiap slot pada hari Sabtu menambah penalty 0.5
        if day.lower() == "sabtu":
            saturday_conflicts.append({
                'id_slot': id_slot,
                'teacher': teacher,
                'kelas': kelas,
                'message': "Kelas dijadwalkan pada hari Sabtu"
            })
            penalty += 0.5

    # Cek konflik dosen (hard constraint)
    for (teacher, day), intervals in teacher_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                teacher_conflicts.append({
                    'teacher': teacher,
                    'day': day,
                    'conflict_ids': (prev[2], curr[2])
                })
                penalty += 1

    # Cek konflik ruangan (hard constraint)
    for (room, day), intervals in room_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                room_conflicts.append({
                    'room': room,
                    'day': day,
                    'conflict_ids': (prev[2], curr[2])
                })
                penalty += 1

    score = max(0, max_score - penalty)
    
    return {
        'score': score,
        'penalty': penalty,
        'violations': {
            'hard_teacher': len(teacher_conflicts),
            'hard_room': len(room_conflicts),
            'teacher_preference': len(teacher_preference_conflicts),
            'saturday': len(saturday_conflicts)
        },
        'teacher_conflicts': teacher_conflicts,
        'room_conflicts': room_conflicts,
        'teacher_preference_conflicts': teacher_preference_conflicts,
        'saturday_conflicts': saturday_conflicts
    }

def validate_rooms(schedule):
    room_intervals = defaultdict(list)
    room_conflicts = []
    
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        room = slot['ruang']
        if room == "Online":
            continue
        day = slot['hari']
        id_slot = slot['id_slot']
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        room_intervals[(room, day)].append((start, end, id_slot))
    
    for (room, day), intervals in room_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                room_conflicts.append({
                    'room': room,
                    'day': day,
                    'conflict_ids': (prev[2], curr[2])
                })
    return room_conflicts

# validasi ruangan
def compare_room_usage(schedule):
    room_usage = defaultdict(list)
    
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        room = slot['ruang']
        usage_info = {
            'id_slot': slot['id_slot'],
            'dosen': slot['dosen'],
            'kelas': slot['kelas'],
            'hari': slot['hari'],
            'jam_mulai': slot['jam_mulai'],
            'jam_selesai': slot['jam_selesai']
        }
        room_usage[room].append(usage_info)
    
    for room in room_usage:
        room_usage[room].sort(key=lambda x: (x['hari'], time_to_minutes(x['jam_mulai'])))
    return room_usage

population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    for slot in schedule:
        print(slot)

    print(f"\nFitness Score: {fitness['score']}")
    print(f"Total Penalty: {fitness['penalty']}")
    
    # validasi konflik dosen (hard constraint)
    print("\nDetail Konflik Dosen (id_slot):")
    if not fitness['teacher_conflicts']:
        print(" - Tidak ada konflik dosen.")
    else:
        for conflict in fitness['teacher_conflicts']:
            conflict_ids = conflict['conflict_ids']
            print(f" - Dosen {conflict['teacher']}: Konflik pada id_slot {conflict_ids[0]} dan {conflict_ids[1]}")
    
    # validasi ruangan (hard constraint)
    print("\nValidasi Data Ruangan:")
    room_conflicts = validate_rooms(schedule)
    if not room_conflicts:
        print(" - Data ruangan valid, tidak ada konflik.")
    else:
        for conflict in room_conflicts:
            conflict_ids = conflict['conflict_ids']
            print(f" - Ruang {conflict['room']} pada {conflict['day']}: Konflik pada id_slot {conflict_ids[0]} dan {conflict_ids[1]}")

    # Soft constraint: preferensi dosen
    print("\nSoft Constraint - Preferensi Dosen (Jam 6 sampai 12):")
    if not fitness['teacher_preference_conflicts']:
        print(" - Tidak ada konflik preferensi dosen.")
    else:
        for conflict in fitness['teacher_preference_conflicts']:
            print(f" - Dosen {conflict['teacher']} pada slot {conflict['id_slot']} (jam mulai: {conflict['jam']}): {conflict['message']}")

    # Soft constraint: hari Sabtu
    print("\nSoft Constraint - Hari Sabtu:")
    if not fitness['saturday_conflicts']:
        print(" - Tidak ada kelas pada hari Sabtu.")
    else:
        for conflict in fitness['saturday_conflicts']:
            print(f" - Slot {conflict['id_slot']} - Dosen {conflict['teacher']}, Kelas {conflict['kelas']}: {conflict['message']}")

    # Komparasi Penggunaan Ruangan Berdasarkan Ruangan
    # print("\nKomparasi Penggunaan Ruangan Berdasarkan Ruangan:")
    # room_usage = compare_room_usage(schedule)
    # if not room_usage:
    #     print(" - Tidak ada data penggunaan ruangan.")
    # else:
    #     for room, usages in room_usage.items():
    #         print(f" - Ruang: {room}")
    #         for usage in usages:
    #             print(f"    ID Slot: {usage['id_slot']}, Dosen: {usage['dosen']}, Kelas: {usage['kelas']}, Hari: {usage['hari']}, Jam Mulai: {usage['jam_mulai']}, Jam Selesai: {usage['jam_selesai']}")



Total mata kuliah yang berhasil dimasukkan: 107 / 107

Individu 1:
{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': 'Pemrograman Web', 'dosen': 'Supriyanto, S.T., M.T.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': 'D', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 3, 'mata_kuliah': 'Pemrograman Web', 'dosen': 'Supriyanto, S.T., M.T.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': 'D', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 4, 'mata_kuliah': 'Pemrograman Web', 'dosen': 'Supriyanto, S.T., M.T.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': 'D', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mul

In [67]:
import random
import copy
import math

class GreyWolf:
    def __init__(self, schedule):
        self.schedule = schedule  # Jadwal kuliah
        self.fitness = None       # Nilai fitness
        self.penalty = None       # Total penalty

def initialize_population(pop_size):
    return [GreyWolf(create_random_schedule()) for _ in range(pop_size)]

def gwo_algorithm(pop_size=10, max_iter=20):
    # Inisialisasi populasi
    population = initialize_population(pop_size)
    
    # Hitung fitness awal
    for wolf in population:
        fitness_result = calculate_fitness(wolf.schedule)
        wolf.fitness = fitness_result['score']
        wolf.penalty = fitness_result['penalty']
    
    # Urutkan populasi berdasarkan fitness
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    # Inisialisasi alpha, beta, delta
    alpha = copy.deepcopy(population[0])
    beta = copy.deepcopy(population[1])
    delta = copy.deepcopy(population[2])
    
    # Proses iterasi
    for iter in range(max_iter):
        a = 2 - iter * (2 / max_iter)  # Koefisien a menurun linear
        
        # Update setiap serigala
        for wolf in population:
            # Dapatkan posisi baru berdasarkan alpha, beta, delta
            new_schedule = update_position(wolf.schedule, alpha.schedule, 
                                          beta.schedule, delta.schedule, a)
            
            # Evaluasi fitness baru
            new_fitness = calculate_fitness(new_schedule)
            
            # Update jika lebih baik
            if new_fitness['score'] > wolf.fitness:
                wolf.schedule = new_schedule
                wolf.fitness = new_fitness['score']
                wolf.penalty = new_fitness['penalty']
        
        # Urutkan ulang populasi
        population.sort(key=lambda x: x.fitness, reverse=True)
        
        # Update alpha, beta, delta
        alpha = copy.deepcopy(population[0])
        beta = copy.deepcopy(population[1])
        delta = copy.deepcopy(population[2])
        
        # Print progress
        print(f"Iterasi {iter+1}: Best Fitness = {alpha.fitness}, Penalty = {alpha.penalty}")
    
    return alpha

def update_position(current, alpha, beta, delta, a):
    # Implementasi position update khusus untuk penjadwalan
    # Menggunakan kombinasi operasi crossover dan mutasi terpandu
    new_schedule = []
    
    # Ambil parameter acak untuk setiap posisi
    A1 = 2 * a * random.random() - a
    C1 = 2 * random.random()
    
    A2 = 2 * a * random.random() - a
    C2 = 2 * random.random()
    
    A3 = 2 * a * random.random() - a
    C3 = 2 * random.random()
    
    # Hybrid crossover dengan alpha, beta, delta
    for i in range(len(current)):
        # Pilih operasi berdasarkan koefisien
        if abs(A1) < 1:
            # Eksploitasi: ikuti alpha
            new_schedule.append(crossover_slots(current[i], alpha[i], C1))
        elif abs(A2) < 1:
            # Eksploitasi: ikuti beta
            new_schedule.append(crossover_slots(current[i], beta[i], C2))
        else:
            # Eksplorasi: ikuti delta
            new_schedule.append(crossover_slots(current[i], delta[i], C3))
    
    # Tambahkan mutasi acak
    if random.random() < 0.1:
        new_schedule = mutate_schedule(new_schedule)
    
    return new_schedule

def crossover_slots(slot1, slot2, C):
    # Operasi crossover untuk dua slot
    new_slot = {}
    for key in slot1:
        if random.random() < C:
            new_slot[key] = slot2[key]
        else:
            new_slot[key] = slot1[key]
    return new_slot

def mutate_schedule(schedule):
    # Operasi mutasi acak
    mutated = copy.deepcopy(schedule)
    
    # Pilih 2 slot acak untuk dimutasi
    for _ in range(2):
        idx = random.randint(0, len(mutated)-1)
        mutated[idx] = create_random_slot()
    
    return mutated

def create_random_slot():
    # Fungsi untuk membuat slot acak (sesuaikan dengan kebutuhan)
    # Contoh implementasi sederhana
    return {
        'id_slot': random.randint(1, 100),
        'mata_kuliah': 'Matkul' + str(random.randint(1, 5)),
        'dosen': 'Dosen' + str(random.randint(1, 5)),
        'ruang': 'Ruang' + str(random.randint(1, 5)),
        'hari': random.choice(['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']),
        'jam_mulai': minutes_to_time(random.randint(420, 1020)),  # 07:00-17:00
        'jam_selesai': minutes_to_time(random.randint(420, 1020)),
        'kelas': random.choice(['A', 'B', 'C']),
        'sks': random.randint(1, 3),
        'metode': random.choice(['Online', 'Offline'])
    }

if __name__ == "__main__":
    best_solution = gwo_algorithm(pop_size=10, max_iter=20)
    
    print("\nBest Solution Found:")
    print(f"Fitness: {best_solution.fitness}")
    print(f"Penalty: {best_solution.penalty}")
    
    print("\nSchedule Details:")
    for slot in best_solution.schedule:
        print(slot)


Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'